# neuroNumber
### Version 0.1
The one hidden layer model for a machine learning

In [3]:
import numpy
from scipy.special import expit
import matplotlib.pyplot as plt

In [7]:
class Neural:
    
    def __init__(self, 
                 input_nodes: int,
                 hidden_nodes_1: int, 
                 hidden_nodes_2: int,
                 output_nodes: int, 
                 learn_rate: float):
        
        self.input_nodes = input_nodes
        self.hidden_nodes_1 = hidden_nodes_1
        self.hidden_nodes_2 = hidden_nodes_2
        self.output_nodes = output_nodes
        self.lr = learn_rate
        
        self.wih = numpy.random.normal(
            0.0, 
            pow(self.hidden_nodes_1, -0.5),
            (self.hidden_nodes_1, self.input_nodes)
        )
        
        self.whh = numpy.random.normal(
            0.0, 
            pow(self.hidden_nodes_2, -0.5),
            (self.hidden_nodes_2, self.hidden_nodes_1)
        )
        
        self.who = numpy.random.normal(
            0.0, 
            pow(self.output_nodes, -0.5),
            (self.output_nodes, self.hidden_nodes_2)
        )
        
        self.activation_fn = lambda x: expit(x)
    
    
    def train(self, inputs_list, targets_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        hidden_outputs_1, \
        hidden_outputs_2, \
        final_outputs = self.get_outputs(inputs)
        
        output_errors = targets - final_outputs
        hidden_errors_2 = numpy.dot(self.who.T, output_errors)
        hidden_errors_1 = numpy.dot(self.whh.T, hidden_errors_2)
        
        self.who += self.lr * numpy.dot(
            (output_errors * final_outputs * (1.0 - final_outputs)),
            numpy.transpose(hidden_outputs_2)
            )
        
        self.whh += self.lr * numpy.dot(
            (hidden_errors_2 * hidden_outputs_2 * (1.0 - hidden_outputs_2)),
            numpy.transpose(hidden_outputs_1)
            )
        
        self.wih += self.lr * numpy.dot(
            (hidden_errors_1 * hidden_outputs_1 * (1.0 - hidden_outputs_1)),
            numpy.transpose(inputs)
            )

    
    def query(self, inputs_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        _, _, final_outputs = self.get_outputs(inputs)
        
        return final_outputs
    
    
    def get_outputs(self, inputs):
        hidden_inputs_1 = numpy.dot(self.wih, inputs)
        hidden_outputs_1 = self.activation_fn(hidden_inputs_1)
        
        hidden_inputs_2 = numpy.dot(self.whh, hidden_outputs_1)
        hidden_outputs_2 = self.activation_fn(hidden_inputs_2)
        
        final_inputs = numpy.dot(self.who, hidden_outputs_2)
        final_outputs = self.activation_fn(final_inputs)
        
        return hidden_outputs_1, hidden_outputs_2, final_outputs

In [ ]:
# Training and test data get from here:
# https://pjreddie.com/projects/mnist-in-csv/
# and pun in workdir
# 
# ../
# neuro_number.ipynb
# mnist_test.csv
# mnist_train.csv

In [14]:
training_file_name = "mnist_train.csv"
input_nodes = 784  # 28*28 - sample size
output_nodes = 10  # 10 digits

hidden_nodes_1 = 280
hidden_nodes_2 = 100

learning_rate = 0.01
epochs = 5

with open(training_file_name, 'r') as training_data_file:
    training_data_list = training_data_file.readlines()
    

n = Neural(input_nodes, 
           hidden_nodes_1,
           hidden_nodes_2,
           output_nodes, 
           learning_rate)

max_count = len(training_data_list)
n.data_length = max_count
n.training_file_name = training_file_name
n.epochs = epochs

print("=== TRAINING ===")
print(f"training file = {n.training_file_name}")
print(f"training data length = {n.data_length}")
print("\n--- Model ---")
print(f"input nodes = {n.input_nodes}")
print(f"hidden nodes 1 = {n.hidden_nodes_1}")
print(f"hidden nodes 2 = {n.hidden_nodes_2}")
print(f"output nodes = {n.output_nodes}")
print(f"learning rate = {n.lr}")

for e in range(epochs):
    print(f"\n--- Epoch {e + 1} of {epochs} ---")
    count = 0
    
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        targets = numpy.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        if count%5000 == 0:
            print(f"[{count + 1} of {max_count}] Training ...")
        n.train(inputs, targets)
        count +=1
    
print("\n=== FINISH ===")


=== TRAINING ===
training file = mnist_train.csv
training data length = 60000

--- Model ---
input nodes = 784
hidden nodes 1 = 280
hidden nodes 2 = 100
output nodes = 10
learning rate = 0.03

--- Epoch 1 of 5 ---
[0 of 60000] Training ...
[5000 of 60000] Training ...
[10000 of 60000] Training ...
[15000 of 60000] Training ...
[20000 of 60000] Training ...
[25000 of 60000] Training ...
[30000 of 60000] Training ...
[35000 of 60000] Training ...
[40000 of 60000] Training ...
[45000 of 60000] Training ...
[50000 of 60000] Training ...
[55000 of 60000] Training ...

--- Epoch 2 of 5 ---
[0 of 60000] Training ...
[5000 of 60000] Training ...
[10000 of 60000] Training ...
[15000 of 60000] Training ...
[20000 of 60000] Training ...
[25000 of 60000] Training ...
[30000 of 60000] Training ...
[35000 of 60000] Training ...
[40000 of 60000] Training ...
[45000 of 60000] Training ...
[50000 of 60000] Training ...
[55000 of 60000] Training ...

--- Epoch 3 of 5 ---
[0 of 60000] Training ...
[5000 

In [16]:
test_file_name = "mnist_test.csv"

with open(test_file_name, 'r') as testing_data_file:
    testing_data_list = testing_data_file.readlines()
    

max_count = len(testing_data_list)
correct = 0.50

count = 0
good = 0

print("=== TEST ===")
print(f"test file: {test_file_name}")
print(f"test data: {max_count} units")
print(f"correct value must be > {correct * 100}%")
print("\n--- Model ---")
print(f"input nodes = {n.input_nodes}")
print(f"hidden nodes 1 = {n.hidden_nodes_1}")
print(f"hidden nodes 2 = {n.hidden_nodes_2}")
print(f"output nodes = {n.output_nodes}")
print(f"learning rate = {n.lr}")
print(f"training file = {n.training_file_name}")
print(f"training data length = {n.data_length}")
print(f"training epochs = {n.epochs}")
        
print()

for record in testing_data_list:
    all_values = record.split(',')
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    
    if count%1000 == 0:
        print(f"[{count + 1} of {max_count}] Testing ...")
    
    res = list(n.query(inputs))
    max_value = max(res)
    idx = res.index(max_value)
    number = int(all_values[0])
    if (float(max_value) > correct) and idx == number:
        good += 1
        
    count +=1

print("\n--- Report ---:")
print(f"Correct values is {good} from {max_count} ({good/max_count * 100:.3f}%)")
print("\n=== FINISH ===")

=== TEST ===
test file: mnist_test.csv
test data: 10000 units
correct value must be > 50.0%

--- Model ---
input nodes = 784
hidden nodes 1 = 280
hidden nodes 2 = 100
output nodes = 10
learning rate = 0.03
training file = mnist_train.csv
training data length = 60000
training epochs = 5

[1 of 10000] Testing ...
[1001 of 10000] Testing ...
[2001 of 10000] Testing ...
[3001 of 10000] Testing ...
[4001 of 10000] Testing ...
[5001 of 10000] Testing ...
[6001 of 10000] Testing ...
[7001 of 10000] Testing ...
[8001 of 10000] Testing ...
[9001 of 10000] Testing ...

--- Report ---:
Correct values is 9541 from 10000 (95.410%)

=== FINISH ===


### Efficiency ~95-96%